# L4: Tools for a Customer Outreach Campaign
See <https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/ksejw/tools-for-a-customer-outreach-campaign-(code)>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [7]:
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [11]:
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)

In [12]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [13]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [14]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [16]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [17]:
from crewai_tools import BaseTool

In [18]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [19]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [20]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [21]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [22]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [23]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather all available information about DeepLearningAI to compile a detailed profile and identify potential areas where our solutions can provide value.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/company/deeplearningai
Snippet: DeepLearning.AI is making a world-cl

In [24]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Impact Through Strategic Collaboration

Dear Andrew Ng,

I hope this message finds you well amidst the exciting developments at DeepLearningAI. As an admirer of your visionary leadership and commitment to democratizing AI education, I am thrilled to reach out and explore potential synergies between CrewAI and DeepLearningAI.

I recently came across the inspiring news about DeepLearningAI's recent milestone in launching the "ChatGPT Prompt Engineering for Developers" course. This initiative not only showcases your dedication to equipping millions of learners with essential AI skills but also reflects your innovative approach towards fostering a global community of AI enthusiasts.

At CrewAI, we are deeply impressed by your company's emphasis on practical learning and hands-on projects to bridge the gap between theoretical knowledge and real-world applications. Our AI technology and expertise align seamlessly with DeepLearningAI's mission of empowering individuals to build a foundation in machine learning and AI skills.

I believe that a strategic collaboration between CrewAI and DeepLearningAI could unlock immense value for both organizations. By tailoring AI courses to address specific industry needs and offering specialized training programs for decision-makers, we can collectively drive innovation and excellence in the AI education landscape.

Andrew, I am excited about the possibility of working together to create customized solutions that cater to the evolving demands of the workforce and align with DeepLearningAI's core values. Your commitment to quality training and cutting-edge research resonates deeply with our ethos of fostering collaboration, creativity, and continuous learning.

I look forward to the opportunity to discuss how CrewAI can support DeepLearningAI in achieving its goals and making a lasting impact on the global AI community. Please feel free to reach out to me at your convenience to explore potential collaboration opportunities further.

Thank you for your time and consideration. Here's to a future filled with innovation, growth, and shared success.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

In [25]:
print(2)

2


In [26]:
print(3)

3
